# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Cloud Storage

下記コードでGCPに接続

In [2]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## Google Sheets

ライブラリのインストール

In [5]:
!pip install --upgrade -q gspread

GoogleSheetsの認証

In [6]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## ESTAT_APP_ID

In [3]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [4]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

'statsDataId'に統計表コードを指定

In [ ]:
statsDataId = '0000020201'

メタ情報を取得する関数を作成

In [ ]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## DataFrameの作成

メタ情報をDataFrameに整形する

In [ ]:
def get_cards_dataframe(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)
 
  # STAT_NAME
  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # cat01
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # DataFrame
  df = pd.json_normalize(cat01)[['@code','@name']]
  
  # 統計表情報の追加
  df['statsDataId'] = statsDataId
  df['statsDataName'] = STAT_NAME

  # 列名の変更
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df= df.rename(columns=columns)

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)
  
  
  # 並べ替え
  df = df.reindex(columns=['statsDataId', 'statsDataName', 'categoryCode','categoryName'])
  
  return df


# print(len(category_codes))

In [ ]:
df = get_cards_dataframe(statsDataId)
df

## 統計カード管理情報をSpreadSheetで加工

統計カードの管理情報を SpreadSheetで管理する

前セルで作業したDataFrameをcsvでコピーするなどして作業すること


[statistics-hyogo/統計カード管理](https://docs.google.com/spreadsheets/d/1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4/edit#gid=0)

# コンテンツ作成

## 統計分野一覧

GoogleSheetsから統計分野一覧を取得

In [13]:
def get_statistics_fields():

  # シート情報の設定
  id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
  sheet_name = 'statisticsFields'
  worksheet = gc.open_by_key(id).worksheet(sheet_name)
  
  return worksheet.get_all_records()

In [ ]:
statistics_fields = get_statistics_fields()
print(statistics_fields)

GoogleDriveとGoogleCloudStorageに保存

In [22]:
def save_json(dic,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)
  
  with open(drive_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-hyogo/{}".format(path)
  with open(gcs_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)

In [23]:
dic = get_statistics_fields()
path = "contents/statistics-fields.json"

save_json(dic,path)

## 統計項目一覧

GoogleSheetsから統計項目一覧を取得

In [24]:
def get_statistics_menus():

  # シート情報の設定
  id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
  sheet_name = 'statisticsMenus'
  worksheet = gc.open_by_key(id).worksheet(sheet_name)
  
  return worksheet.get_all_records()

In [ ]:
statistics_menus = get_statistics_menus()
print(statistics_menus)

GoogleDriveとGoogleCloudStorageに保存

In [26]:
def save_json(dic,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)
  
  with open(drive_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-hyogo/{}".format(path)
  with open(gcs_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)

In [27]:
dic = get_statistics_menus()
path = "contents/statistics-menus.json"

save_json(dic,path)

## 統計カード一覧

GoogleSheetsから統計カード一覧を取得

In [105]:
import numpy as np

def get_statistics_cards():

  # シート情報の設定
  id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
  sheet_name = 'statisticsCards'
  worksheet = gc.open_by_key(id).worksheet(sheet_name)

  # データを取得してDataFrameに格納
  dic = worksheet.get_all_records()
  df = pd.DataFrame(dic)

  # 必要な列だけ抽出
  df =  df[['cardId','cardTitle','cardIndex','governmentType','menuId','menuTitle','fieldId','fieldTitle']]
  
  # 空白行を削除
  df['cardId'].replace('', np.nan, inplace=True)
  df.dropna(subset=['cardId'], inplace=True)

  # 重複を削除
  df = df.drop_duplicates(subset='cardId')

  return df

In [ ]:
df_statistics_cards = get_statistics_cards()
df_statistics_cards

GoogleDriveとGoogleCloudStorageに保存

In [107]:
def save_json(df,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)
  df.reset_index().to_json(drive_path,orient='records',force_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-hyogo/{}".format(path)
  df.reset_index().to_json(gcs_path,orient='records',force_ascii=False)

In [108]:
df = get_statistics_cards()
path = "contents/statistics-cards.json"

save_json(df,path)

## routes

都道府県一覧

In [118]:
def get_preflist():
  with open('drive/MyDrive/statistics-hyogo/resas/preflist.json') as f:
    return json.load(f)

市区町村一覧

In [ ]:
def get_citylist():
  with open('drive/MyDrive/statistics-hyogo/resas/citylist.json') as f:
    return json.load(f)

print(get_citylist())

統計カード一覧

In [ ]:
def get_statistics_cards():
  with open('drive/MyDrive/statistics-hyogo/contents/statistics-cards.json') as f:
    return json.load(f)

print(get_statistics_cards())

routes一覧を作成

In [138]:
def generate_routes(cards):

  res = []
  for card in cards:
    
    # 値の取得
    governmentType = card['governmentType']
    fieldId = card['fieldId']
    menuId = card['menuId']
    cardId = card['cardId']
  
    # 地域コード一覧作成
    codes = []
    if governmentType == 'prefecture':
      codes = [d.get('prefCode') for d in get_preflist()]
      codes = [f'{n:02}'+'000' for n in codes]
    elif governmentType == 'city':
      codes = [d.get('cityCode') for d in get_citylist()]

    # routes作成
    for code in codes:
      route = '/{}/{}/{}/{}/{}/'.format(governmentType,code,fieldId,menuId,cardId)
      res.append(route)

  return res


In [ ]:
statistics_cards = get_statistics_cards()
routes = generate_routes(statistics_cards)
print(routes)

GoogleDriveとGoogleCloudStorageに保存

In [142]:
def save_json(dic,path):
  
  # Google Driveへ保存
  drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)
  
  with open(drive_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)
  
  # GoogleCloudStorage
  gcs_path = "statistics-hyogo/{}".format(path)
  with open(gcs_path, 'w') as f:
    json.dump(dic,f,ensure_ascii=False)

In [143]:
routes = generate_routes(statistics_cards)
path = "routes/routes.json"

save_json(routes,path)

## sitemap.xml

サイトマップに反映させるroutesは兵庫県だけ

In [147]:
def generate_routes_sitemap(cards):

  res = []
  for card in cards:
    
    # 値の取得
    governmentType = card['governmentType']
    fieldId = card['fieldId']
    menuId = card['menuId']
    cardId = card['cardId']

    # 地域コード一覧作成
    codes = []
    if governmentType == 'prefecture':
      codes = ['28000']
    elif governmentType == 'city':
      citylist = list(filter(lambda x: x['prefCode'] == 28, get_citylist()))
      codes = [d.get('cityCode') for d in citylist]

    # routes作成
    for code in codes:
      route = '/{}/{}/{}/{}/{}/'.format(governmentType,code,fieldId,menuId,cardId)
      res.append(route)

  return res

In [148]:
statistics_cards = get_statistics_cards()
routes_sitemap = generate_routes_sitemap(statistics_cards)
print(routes_sitemap)

['/prefecture/28000/population/population/total-population-prefecture/', '/prefecture/28000/population/marriage/unmarried-prefecture/', '/prefecture/28000/population/marriage/spouse-prefecture/', '/prefecture/28000/population/marriage/separated-prefecture/', '/prefecture/28000/population/death/standardized-mortality-prefecture/', '/prefecture/28000/population/household/household-prefecture/', '/prefecture/28000/population/household/single-mother-household-prefecture/', '/prefecture/28000/population/marriage/first-marriage-age-prefecture/', '/prefecture/28000/population/marriage/divorce-prefecture/', '/city/28100/population/population/total-population-city/', '/city/28101/population/population/total-population-city/', '/city/28102/population/population/total-population-city/', '/city/28105/population/population/total-population-city/', '/city/28106/population/population/total-population-city/', '/city/28107/population/population/total-population-city/', '/city/28108/population/populatio

sitemap.xmlを保存

In [ ]:
import xml.etree.ElementTree as ET
import datetime
import tempfile

def generate_sitemap(path):

    urlset = ET.Element('urlset')
    urlset.set('xmlns', 'http://www.sitemaps.org/schemas/sitemap/0.9')
    urlset.set('xmlns:news', 'http://www.google.com/schemas/sitemap-news/0.9')
    urlset.set('xmlns:xhtml', 'http://www.w3.org/1999/xhtml')
    urlset.set('xmlns:mobile', 'http://www.google.com/schemas/sitemap-mobile/1.0')
    urlset.set('xmlns:image', 'http://www.google.com/schemas/sitemap-image/1.1')
    urlset.set('xmlns:video', 'http://www.google.com/schemas/sitemap-video/1.1')
    tree = ET.ElementTree(element=urlset)

    for route in routes():
        url = 'https://statistics-hyogo.com'+route
        updated = datetime.date.today()
        updated = updated.strftime('%Y-%m-%d')
        url_element = ET.SubElement(urlset, 'url')
        loc = ET.SubElement(url_element, 'loc')
        loc.text = url
        lastmod = ET.SubElement(url_element, 'lastmod')
        lastmod.text = updated
        
    _, temp_local_filename = tempfile.mkstemp()
    tree.write(temp_local_filename, encoding='utf-8', xml_declaration=True)
        
        
    # bucket = storage_client.bucket('statistics-hyogo')
    # blob = bucket.blob('sitemap.xml')
    # blob.upload_from_filename(temp_local_filename)
    # os.remove(temp_local_filename)

    return 'sitemap.xml generated !!'

# BigQueryへ保存

ライブラリのインストール

In [149]:
!pip install --upgrade -q gspread

SpreadSheetの認証

In [150]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheetを読み込んでDataFrameに変換

In [ ]:
# シート情報の設定
id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
sheet_name = 'statisticsCards'
worksheet = gc.open_by_key(id).worksheet(sheet_name)

# DataFrameに格納
dic = worksheet.get_all_records(head=1)
df = pd.DataFrame(dic)
df

pandas-gbqのインストール

In [ ]:
!pip install pandas-gbq

DataFrameをBigQueryへ保存

In [ ]:
import pandas as pd

# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'contents'
table_id='cards'

# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")